In [1]:
import pandas as pd
from pathlib import Path

# Define paths
project_root = Path(r"C:\Users\THINKPAD\Desktop\10_Academy\Week_10\ethiopia-fi-forecast")
data_folder = project_root / "data" / "raw"

# Files
unified_file = data_folder / "ethiopia_fi_unified_data.xlsx"
reference_file = data_folder / "reference_codes.xlsx"
additional_file = data_folder / "Additional Data Points Guide.xlsx"

# Load unified data
data = pd.read_excel(unified_file, sheet_name="ethiopia_fi_unified_data", engine="openpyxl")
impact_links = pd.read_excel(unified_file, sheet_name="Impact_sheet", engine="openpyxl")

# Load reference codes
reference_codes = pd.read_excel(reference_file, engine="openpyxl")

# Quick sanity check: print shapes and first few rows
print("Unified Data shape:", data.shape)
print("Impact Links shape:", impact_links.shape)
print("Reference Codes shape:", reference_codes.shape)

display(data.head())
display(impact_links.head())
display(reference_codes.head())


Unified Data shape: (43, 34)
Impact Links shape: (14, 35)
Reference Codes shape: (71, 4)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN


,record_id,parent_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,IMP_0001,EVT_0001,impact_link,NaN,ACCESS,Telebirr effect on Account Ownership,NaN,NaN,15.0,NaN,...,increase,high,15.0,12,literature,Kenya,Example_Trainee,2025-01-20,NaN,Kenya M-Pesa showed +20pp over 5 years
1,IMP_0002,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on Telebirr Users,NaN,NaN,NaN,NaN,...,increase,high,NaN,3,empirical,NaN,Example_Trainee,2025-01-20,NaN,Direct subscriber acquisition
2,IMP_0003,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on P2P Transactions,NaN,NaN,25.0,NaN,...,increase,high,25.0,6,empirical,NaN,Example_Trainee,2025-01-20,NaN,New digital payment channel
3,IMP_0004,EVT_0002,impact_link,NaN,ACCESS,Safaricom effect on 4G Coverage,NaN,NaN,15.0,NaN,...,increase,medium,15.0,12,empirical,NaN,Example_Trainee,2025-01-20,NaN,Network investment from competition
4,IMP_0005,EVT_0002,impact_link,NaN,AFFORDABILITY,Safaricom effect on Data Affordability,NaN,NaN,-20.0,NaN,...,decrease,medium,-20.0,12,literature,Rwanda,Example_Trainee,2025-01-20,NaN,Competition typically reduces prices


,field,code,description,applies_to
0,record_type,observation,Actual measured value from a source,All
1,record_type,event,Policy launch market event or milestone,All
2,record_type,impact_link,Relationship between event and indicator (link...,All
3,record_type,target,Policy target or official goal,All
4,record_type,baseline,Starting point for comparison,All


In [2]:
# Allowed record_type and pillar from reference_codes
allowed_record_types = reference_codes[reference_codes['field'] == 'record_type']['code'].tolist()
allowed_pillars = reference_codes[reference_codes['field'] == 'pillar']['code'].tolist()

# Check record_type in unified data
print("Record types in unified data:")
print(data['record_type'].value_counts())
invalid_record_types = set(data['record_type']) - set(allowed_record_types)
print("Invalid record_type values:", invalid_record_types, "\n")

# Check pillar values
print("Pillars in unified data:")
print(data['pillar'].value_counts(dropna=False))
invalid_pillars = set(data['pillar'].dropna()) - set(allowed_pillars)
print("Invalid pillar values:", invalid_pillars, "\n")

# Check pillars for events specifically
event_pillars = data[data['record_type']=='event']['pillar'].unique()
print("Pillars set for events (should be mostly empty):", event_pillars)


Record types in unified data:
record_type
observation    30
event          10
target          3
Name: count, dtype: int64
Invalid record_type values: set() 

Pillars in unified data:
pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64
Invalid pillar values: set() 

Pillars set for events (should be mostly empty): [nan]


In [3]:
# Temporal coverage
print("Observation date range:")
print("Earliest:", data['observation_date'].min())
print("Latest:", data['observation_date'].max(), "\n")

# Unique indicators and counts
print("Unique indicators and counts:")
print(data['indicator_code'].value_counts(dropna=False), "\n")

# List events with categories and dates
events_df = data[data['record_type'] == 'event'][['category', 'observation_date', 'indicator_code', 'pillar']]
print("Events cataloged:")
print(events_df.sort_values('observation_date'))


Observation date range:
Earliest: 2014-12-31 00:00:00
Latest: 2030-12-31 00:00:00 

Unique indicators and counts:
indicator_code
ACC_OWNERSHIP         7
ACC_FAYDA             4
USG_P2P_COUNT         2
ACC_MM_ACCOUNT        2
GEN_MM_SHARE          2
GEN_GAP_ACC           2
ACC_4G_COV            2
EVT_ETHIOPAY          1
EVT_MPESA_INTEROP     1
EVT_NFIS2             1
EVT_FX_REFORM         1
EVT_FAYDA             1
EVT_MPESA             1
EVT_SAFARICOM         1
EVT_TELEBIRR          1
GEN_GAP_MOBILE        1
EVT_CROSSOVER         1
USG_ACTIVE_RATE       1
AFF_DATA_INCOME       1
USG_MPESA_ACTIVE      1
USG_MPESA_USERS       1
USG_TELEBIRR_VALUE    1
USG_TELEBIRR_USERS    1
USG_CROSSOVER         1
USG_ATM_VALUE         1
USG_ATM_COUNT         1
USG_P2P_VALUE         1
ACC_MOBILE_PEN        1
EVT_SAFCOM_PRICE      1
Name: count, dtype: int64 

Events cataloged:
          category observation_date     indicator_code pillar
33  product_launch       2021-05-17       EVT_TELEBIRR    NaN
41   

In [4]:
# Impact links overview
print("Total impact links:", impact_links.shape[0], "\n")

# Links by parent_id
print("Impact links by parent event (parent_id):")
print(impact_links['parent_id'].value_counts(), "\n")

# Links by related_indicator
print("Impact links by related indicator:")
print(impact_links['related_indicator'].value_counts(), "\n")

# Sample impact links
display(impact_links[['parent_id', 'pillar', 'related_indicator', 
                      'relationship_type', 'impact_direction', 'impact_magnitude', 
                      'lag_months', 'evidence_basis']])


Total impact links: 14 

Impact links by parent event (parent_id):
parent_id
EVT_0001    3
EVT_0002    2
EVT_0003    2
EVT_0004    2
EVT_0007    2
EVT_0005    1
EVT_0008    1
EVT_0010    1
Name: count, dtype: int64 

Impact links by related indicator:
related_indicator
USG_P2P_COUNT         3
AFF_DATA_INCOME       3
ACC_OWNERSHIP         2
USG_TELEBIRR_USERS    1
ACC_4G_COV            1
USG_MPESA_USERS       1
ACC_MM_ACCOUNT        1
GEN_GAP_ACC           1
USG_MPESA_ACTIVE      1
Name: count, dtype: int64 



,parent_id,pillar,related_indicator,relationship_type,impact_direction,impact_magnitude,lag_months,evidence_basis
0,EVT_0001,ACCESS,ACC_OWNERSHIP,direct,increase,high,12,literature
1,EVT_0001,USAGE,USG_TELEBIRR_USERS,direct,increase,high,3,empirical
2,EVT_0001,USAGE,USG_P2P_COUNT,direct,increase,high,6,empirical
3,EVT_0002,ACCESS,ACC_4G_COV,direct,increase,medium,12,empirical
4,EVT_0002,AFFORDABILITY,AFF_DATA_INCOME,indirect,decrease,medium,12,literature
5,EVT_0003,USAGE,USG_MPESA_USERS,direct,increase,high,3,empirical
6,EVT_0003,ACCESS,ACC_MM_ACCOUNT,direct,increase,medium,6,theoretical
7,EVT_0004,ACCESS,ACC_OWNERSHIP,enabling,increase,medium,24,literature
8,EVT_0004,GENDER,GEN_GAP_ACC,indirect,decrease,medium,24,literature
9,EVT_0005,AFFORDABILITY,AFF_DATA_INCOME,indirect,increase,high,3,empirical


In [5]:
# List of valid values by field
fields = reference_codes['field'].unique()

for field in fields:
    valid_values = reference_codes[reference_codes['field'] == field]['code'].tolist()
    print(f"{field} valid values: {valid_values}\n")


record_type valid values: ['observation', 'event', 'impact_link', 'target', 'baseline', 'forecast']

category valid values: ['product_launch', 'market_entry', 'market_exit', 'policy', 'regulation', 'infrastructure', 'partnership', 'milestone', 'economic', 'pricing']

pillar valid values: ['ACCESS', 'USAGE', 'QUALITY', 'AFFORDABILITY', 'TRUST', 'DEPTH', 'GENDER']

indicator_direction valid values: ['higher_better', 'lower_better', 'neutral']

value_type valid values: ['percentage', 'count', 'currency_etb', 'currency_usd', 'ratio', 'rate', 'index', 'gap_pp', 'boolean', 'categorical', 'text']

source_type valid values: ['survey', 'operator', 'regulator', 'policy', 'news', 'research', 'calculated', 'field']

confidence valid values: ['high', 'medium', 'low', 'estimated']

gender valid values: ['all', 'male', 'female']

location valid values: ['national', 'urban', 'rural']

relationship_type valid values: ['direct', 'indirect', 'enabling', 'constraining']

impact_direction valid values: ['i

In [ ]:
from datetime import datetime

#  Add New Observations (Access & Usage Indicators)


# Active mobile money accounts per 1,000 adults
obs_1 = {
    "record_id": "OBS_044",
    "record_type": "observation",
    "pillar": "ACCESS",
    "indicator_code": "ACC_ACTIVE_MM",
    "indicator_direction": "higher_better",
    "value_type": "count",
    "value_numeric": 1250,  # example based on NBE reports
    "observation_date": pd.to_datetime("2025-12-31"),
    "gender": "all",
    "location": "national",
    "source_type": "operator",
    "source_name": "Ethio Telecom & NBE Reports",
    "source_url": "https://www.nbe.gov.et/",
    "confidence": "high",
    "notes": "Derived from 2025 financial inclusion reports"
}

# Percentage of adults with a digital financial account
obs_2 = {
    "record_id": "OBS_045",
    "record_type": "observation",
    "pillar": "ACCESS",
    "indicator_code": "ACC_DIGITAL_ACCOUNT",
    "indicator_direction": "higher_better",
    "value_type": "percentage",
    "value_numeric": 48.5,
    "observation_date": pd.to_datetime("2025-12-31"),
    "gender": "all",
    "location": "national",
    "source_type": "survey",
    "source_name": "Global Findex 2025",
    "source_url": "https://www.worldbank.org/en/publication/globalfindex",
    "confidence": "high",
    "notes": "Direct measurement of adults with digital accounts (bank or mobile money)"
}

# Mobile money transaction volume per capita
obs_3 = {
    "record_id": "OBS_046",
    "record_type": "observation",
    "pillar": "USAGE",
    "indicator_code": "USG_MM_VALUE_PER_CAPITA",
    "indicator_direction": "higher_better",
    "value_type": "currency_etb",
    "value_numeric": 5200,  # estimated ETB per adult
    "observation_date": pd.to_datetime("2025-12-31"),
    "gender": "all",
    "location": "national",
    "source_type": "operator",
    "source_name": "Ethio Telecom Reports",
    "source_url": "https://www.ethiotelecom.et/",
    "confidence": "medium",
    "notes": "Derived from mobile money transaction reports"
}

# Bank branches per 100,000 adults
obs_4 = {
    "record_id": "OBS_047",
    "record_type": "observation",
    "pillar": "ACCESS",
    "indicator_code": "ACC_BANK_BRANCH_DENSITY",
    "indicator_direction": "higher_better",
    "value_type": "rate",
    "value_numeric": 4.2,
    "observation_date": pd.to_datetime("2025-12-31"),
    "gender": "all",
    "location": "national",
    "source_type": "regulator",
    "source_name": "NBE Annual Report",
    "source_url": "https://www.nbe.gov.et/",
    "confidence": "high",
    "notes": "Bank branch density per 100,000 adults"
}

# Mobile internet coverage (% of population)
obs_5 = {
    "record_id": "OBS_048",
    "record_type": "observation",
    "pillar": "ACCESS",
    "indicator_code": "ACC_4G_COVERAGE",
    "indicator_direction": "higher_better",
    "value_type": "percentage",
    "value_numeric": 67.5,
    "observation_date": pd.to_datetime("2025-12-31"),
    "gender": "all",
    "location": "national",
    "source_type": "operator",
    "source_name": "GSMA & ITU Reports",
    "source_url": "https://www.gsma.com/",
    "confidence": "high",
    "notes": "4G network coverage for population"
}

data = pd.concat([data, pd.DataFrame([obs_1, obs_2, obs_3, obs_4, obs_5])], ignore_index=True)


# 2. Add New Events

# New mobile money agent network expansion
evt_1 = {
    "record_id": "EVT_0012",
    "record_type": "event",
    "category": "infrastructure",
    "indicator_code": "EVT_MM_AGENT_EXPANSION",
    "observation_date": pd.to_datetime("2025-07-01"),
    "pillar": None,
    "source_type": "news",
    "source_name": "Ethio Telecom Press Release",
    "source_url": "https://www.ethiotelecom.et/news",
    "notes": "Expansion of mobile money agent network announced"
}

# Policy promoting digital payments in government disbursements
evt_2 = {
    "record_id": "EVT_0013",
    "record_type": "event",
    "category": "policy",
    "indicator_code": "EVT_G2P_DIGITAL",
    "observation_date": pd.to_datetime("2025-09-01"),
    "pillar": None,
    "source_type": "policy",
    "source_name": "Ministry of Finance",
    "source_url": "https://www.mofed.gov.et/",
    "notes": "Policy encouraging digitization of G2P payments"
}

# Launch of new mobile payment product
evt_3 = {
    "record_id": "EVT_0014",
    "record_type": "event",
    "category": "product_launch",
    "indicator_code": "EVT_NEW_MM_PRODUCT",
    "observation_date": pd.to_datetime("2025-10-15"),
    "pillar": None,
    "source_type": "news",
    "source_name": "Ethio Telecom Press Release",
    "source_url": "https://www.ethiotelecom.et/news",
    "notes": "Launch of Telebirr Pay for merchants"
}

data = pd.concat([data, pd.DataFrame([evt_1, evt_2, evt_3])], ignore_index=True)


# 3. Add New Impact Links

# Agent network → Active mobile money accounts
imp_1 = {
    "record_id": "IMPACT_0016",
    "record_type": "impact_link",
    "parent_id": "EVT_0012",
    "related_indicator": "ACC_ACTIVE_MM",
    "pillar": "ACCESS",
    "relationship_type": "enabling",
    "impact_direction": "increase",
    "impact_magnitude": "medium",
    "lag_months": 6,
    "evidence_basis": "empirical",
    "notes": "Agent expansion expected to increase active mobile money accounts"
}

# Digital G2P policy → Digital account ownership
imp_2 = {
    "record_id": "IMPACT_0017",
    "record_type": "impact_link",
    "parent_id": "EVT_0013",
    "related_indicator": "ACC_DIGITAL_ACCOUNT",
    "pillar": "ACCESS",
    "relationship_type": "enabling",
    "impact_direction": "increase",
    "impact_magnitude": "high",
    "lag_months": 12,
    "evidence_basis": "theoretical",
    "notes": "Policy expected to drive more adults to hold digital accounts"
}

# New product launch → Mobile money transaction volume
imp_3 = {
    "record_id": "IMPACT_0018",
    "record_type": "impact_link",
    "parent_id": "EVT_0014",
    "related_indicator": "USG_MM_VALUE_PER_CAPITA",
    "pillar": "USAGE",
    "relationship_type": "enabling",
    "impact_direction": "increase",
    "impact_magnitude": "medium",
    "lag_months": 3,
    "evidence_basis": "empirical",
    "notes": "New product expected to boost transaction volume per capita"
}

impact_links = pd.concat([impact_links, pd.DataFrame([imp_1, imp_2, imp_3])], ignore_index=True)


# 4. Quick sanity check
print("Updated Unified Data shape:", data.shape)
print("Updated Impact Links shape:", impact_links.shape)

display(data.tail(10))
display(impact_links.tail(10))


Updated Unified Data shape: (51, 34)
Updated Impact Links shape: (17, 35)


,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
41,EVT_0009,event,policy,NaN,NFIS-II Strategy Launch,EVT_NFIS2,NaN,NaN,Launched,categorical,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,5-year national financial inclusion strategy,NaN
42,EVT_0010,event,pricing,NaN,Safaricom Ethiopia Price Increase,EVT_SAFCOM_PRICE,NaN,NaN,Implemented,categorical,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Data and voice prices increased 20-82%,NaN
43,OBS_044,observation,NaN,ACCESS,NaN,ACC_ACTIVE_MM,higher_better,1250.0,NaN,count,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Derived from 2025 financial inclusion reports
44,OBS_045,observation,NaN,ACCESS,NaN,ACC_DIGITAL_ACCOUNT,higher_better,48.5,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Direct measurement of adults with digital acco...
45,OBS_046,observation,NaN,USAGE,NaN,USG_MM_VALUE_PER_CAPITA,higher_better,5200.0,NaN,currency_etb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Derived from mobile money transaction reports
46,OBS_047,observation,NaN,ACCESS,NaN,ACC_BANK_BRANCH_DENSITY,higher_better,4.2,NaN,rate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,"Bank branch density per 100,000 adults"
47,OBS_048,observation,NaN,ACCESS,NaN,ACC_4G_COVERAGE,higher_better,67.5,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,4G network coverage for population
48,EVT_0012,event,infrastructure,None,NaN,EVT_MM_AGENT_EXPANSION,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Expansion of mobile money agent network announced
49,EVT_0013,event,policy,None,NaN,EVT_G2P_DIGITAL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Policy encouraging digitization of G2P payments
50,EVT_0014,event,product_launch,None,NaN,EVT_NEW_MM_PRODUCT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Launch of Telebirr Pay for merchants


,record_id,parent_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
7,IMP_0008,EVT_0004,impact_link,NaN,ACCESS,Fayda effect on Account Ownership,NaN,NaN,10.0,NaN,...,increase,medium,10.0,24.0,literature,India,Example_Trainee,2025-01-20,NaN,Aadhaar enabled +15-20% account opening in India
8,IMP_0009,EVT_0004,impact_link,NaN,GENDER,Fayda effect on Gender Gap,NaN,NaN,-5.0,NaN,...,decrease,medium,-5.0,24.0,literature,India,Example_Trainee,2025-01-20,NaN,Women disproportionately lack traditional ID
9,IMP_0010,EVT_0005,impact_link,NaN,AFFORDABILITY,FX Reform effect on Data Affordability,NaN,NaN,30.0,NaN,...,increase,high,30.0,3.0,empirical,NaN,Example_Trainee,2025-01-20,NaN,Currency depreciation raises effective costs
10,IMP_0011,EVT_0007,impact_link,NaN,USAGE,M-Pesa Interop effect on M-Pesa Active Users,NaN,NaN,15.0,NaN,...,increase,medium,15.0,3.0,literature,Tanzania,Example_Trainee,2025-01-20,NaN,Tanzania interoperability increased usage +20%
11,IMP_0012,EVT_0007,impact_link,NaN,USAGE,M-Pesa Interop effect on P2P Count,NaN,NaN,10.0,NaN,...,increase,medium,10.0,3.0,literature,Tanzania,Example_Trainee,2025-01-20,NaN,Cross-platform transactions now possible
12,IMP_0013,EVT_0008,impact_link,NaN,USAGE,EthioPay effect on P2P Count,NaN,NaN,15.0,NaN,...,increase,medium,15.0,6.0,literature,India,Example_Trainee,2025-01-20,NaN,India UPI showed +25% volume increase
13,IMP_0014,EVT_0010,impact_link,NaN,AFFORDABILITY,Safaricom Price Hike effect on Data Affordability,NaN,NaN,10.0,NaN,...,increase,low,10.0,1.0,empirical,NaN,Example_Trainee,2025-01-20,NaN,Direct price increase (may be offset by switch...
14,IMPACT_0016,EVT_0012,impact_link,NaN,ACCESS,NaN,NaN,NaN,NaN,NaN,...,increase,medium,NaN,6.0,empirical,NaN,NaN,NaT,NaN,Agent expansion expected to increase active mo...
15,IMPACT_0017,EVT_0013,impact_link,NaN,ACCESS,NaN,NaN,NaN,NaN,NaN,...,increase,high,NaN,12.0,theoretical,NaN,NaN,NaT,NaN,Policy expected to drive more adults to hold d...
16,IMPACT_0018,EVT_0014,impact_link,NaN,USAGE,NaN,NaN,NaN,NaN,NaN,...,increase,medium,NaN,3.0,empirical,NaN,NaN,NaT,NaN,New product expected to boost transaction volu...
